In [2]:
from ultralytics import YOLO
import cv2

# Cargar el modelo preentrenado (por ejemplo, YOLOv8n, YOLOv8s, etc.)
model = YOLO("yolov8n.pt") 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/dani/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:01<00:00, 5.57MB/s]


In [45]:
# Cargar imagen
image_path = "example.jpg"
results = model(image_path)
print(results)  # Imprimir resultados

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant'

In [46]:
# Filtrar personas y mostrar resultados
for result in results:
    boxes = result.boxes
    for box in boxes:
        cls_id = int(box.cls[0])  # Clase detectada
        if model.names[cls_id] == 'person':
            x1, y1, x2, y2 = box.xyxy[0].tolist()  # Coordenadas de la caja
            print(f"Persona detectada en: ({x1}, {y1}), ({x2}, {y2})")

Persona detectada en: (112.66088104248047, 18.494600296020508), (424.00311279296875, 377.3894958496094)
Persona detectada en: (34.22816848754883, 84.89447021484375), (108.04837036132812, 295.0907897949219)
Persona detectada en: (537.9710083007812, 125.1525650024414), (592.1952514648438, 233.28982543945312)
Persona detectada en: (371.10272216796875, 67.38673400878906), (427.9478759765625, 314.1593933105469)
Persona detectada en: (117.50711822509766, 122.63914489746094), (219.3166046142578, 310.6016845703125)


In [47]:
# Mostrar la imagen con las detecciones
results[0].plot()  # Dibuja las cajas
cv2.imshow("Detecciones", results[0].plot())
cv2.waitKey(0)
cv2.destroyAllWindows()

# Video

In [36]:
from ultralytics import YOLO
import cv2
import torch
import numpy as np
from collections import Counter

print("GPU disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Usando:", torch.cuda.get_device_name(0))

model = YOLO("yolov8n.pt")
model_line = YOLO("yolov8n_line_detail.pt")  
model.to('cuda' if torch.cuda.is_available() else 'cpu')
model_line.to('cuda' if torch.cuda.is_available() else 'cpu')
print("Modelo cargado en:", model.device)

GPU disponible: True
Usando: NVIDIA GeForce RTX 4060
Modelo cargado en: cuda:0


In [51]:
cap = cv2.VideoCapture("SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_720p.mkv")


if not cap.isOpened():
    print("No se pudo abrir el video.")
    exit()
# Rango para el color blanco
lower_white = np.array([30, 10, 210])
upper_white = np.array([110, 70, 255])
prev_frame = None
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    lines = model_line.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    annotated_frame = frame.copy()
    for result in results:
        boxes = result.boxes
        detections = []
        for box in boxes:
            cls_id = int(box.cls[0])  # Clase detectada
            """ if model.names[cls_id] == 'person':
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                conf = float(box.conf[0])
                detections.append(([x1, y1, x2, y2], conf, cls_id))
            elif model.names[cls_id] == 'sports ball':
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                conf = float(box.conf[0])
                detections.append(([x1, y1, x2, y2], conf, cls_id))
    
        tracks = tracker.update_tracks(detections, frame=frame)
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            x1, y1, x2, y2 = track.to_tlbr()
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
            # Dibuja la caja delimitadora y el ID de seguimiento
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) """
            if model.names[cls_id] == 'person':
                x1, y1, x2, y2 = box.xyxy[0].tolist()  
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                person_section = annotated_frame[y1:y2, x1:x2]
                small_section = cv2.resize(person_section, (20, 20), interpolation=cv2.INTER_LINEAR)
                hsv_section = cv2.cvtColor(small_section, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv_section, np.array([35, 40, 40]), np.array([85, 255, 255]))
                non_green_pixels = hsv_section[mask == 0]
                quantized_pixels = (non_green_pixels // 32) * 32
                pixels = quantized_pixels.reshape(-1, 3)
                pixels_list = [tuple(pixel) for pixel in pixels]
                if len(pixels_list) > 0:
                    most_common = Counter(pixels_list).most_common(1)[0][0]
                    domimant_color_rgb = cv2.cvtColor(np.uint8([[most_common]]), cv2.COLOR_HSV2BGR)[0][0]
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), tuple(int(c) for c in domimant_color_rgb), 2)
                else:
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            #balon
            if model.names[cls_id] == 'sports ball':
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
    # Dibuja las líneas detectadas
    field_lines = []
    for line in lines:
        boxes = line.boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            if model_line.names[cls_id] in ['Circle right', 'Circle central', 'Circle left']:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                #creamos una linea entre los puntos
                #cv2.line(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                #anotamos la clase
                cv2.putText(annotated_frame, model_line.names[cls_id], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            elif model_line.names[cls_id] in ['goal', 'unknown']:
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                #anotamos la clase
                cv2.putText(annotated_frame, model_line.names[cls_id], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            else:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                #vamos a hacer deteccion de bordes en esta zona
                roi = annotated_frame[y1:y2, x1:x2]
                # Convertir a escala de grises
                gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                # Aplicar un filtro de suavizado
                blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)
                # Aplicar Canny para detectar bordes
                edges = cv2.Canny(blurred_roi, 50, 150)
                # mostrar los bordes
                lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=20, maxLineGap=10)

                if lines is not None:
                    angles = []
                    for line in lines:
                        x1_line, y1_line, x2_line, y2_line = line[0]
                        # Calcular dirección en grados
                        angle_rad = np.arctan2((y2_line - y1_line), (x2_line - x1_line))
                        angle_deg = np.degrees(angle_rad)
                        angles.append(angle_deg)
                        #cv2.line(annotated_frame, (x1 + x1_line, y1 + y1_line), (x1 + x2_line, y1 + y2_line), (0, 255, 255), 2)
                    # Calcular el ángulo promedio
                    if len(angles) > 0:
                        avg_angle = np.mean(angles)
                        print(f"Ángulo promedio: {avg_angle:.2f} grados, {model_line.names[cls_id]}")
                        if avg_angle > 0:
                            cv2.line(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        else:
                            cv2.line(annotated_frame, (x1, y2), (x2, y1), (255, 0, 0), 2)
                #cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                #anotamos la clase
                cv2.putText(annotated_frame, model_line.names[cls_id], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                field_lines.append([x1, y1, x2, y2])                      
                    
    #Mostrar mascara
    #cv2.imshow("Mascara", masked_frame)
    #cv2.imshow("Bordes del Video", edges)
    # Mostrar la imagen con las detecciones
    cv2.imshow("Detecciones", annotated_frame)
    # Esperar hasta que se presione una tecla
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Ángulo promedio: -30.28 grados, Middle line
Ángulo promedio: -50.00 grados, Middle line
Ángulo promedio: -38.79 grados, Middle line
Ángulo promedio: -30.32 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -30.16 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -30.14 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -89.72 grados, Middle line
Ángulo promedio: 0.09 grados, Side line top
Ángulo promedio: -16.01 grados, Big rect. left main
Ángulo promedio: 0.07 grados, Side line top
Ángulo promedio: 0.21 grados, Side line top
Ángulo promedio: -16.41 grados, Big rect. left main
Ángulo promedio: 0.27 grados, Side line top
Ángulo promedio: -16.79 grados, Big rect. left main
Ángulo p